<a href="https://colab.research.google.com/github/samsiroos/SLM-Architecture/blob/main/%D8%A2%D9%85%D9%88%D8%B2%D8%B4_%D9%88_%D8%AA%D9%88%D9%84%DB%8C%D8%AF_%D9%85%D8%AA%D9%86_%D8%A8%D8%A7_GRU_%D8%AF%D8%B1_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# وارد کردن کتابخانه‌های لازم
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np

print("کتابخانه‌ها با موفقیت وارد شدند.")

# --- ۱. آماده‌سازی داده‌ها ---
# متن نمونه طولانی‌تر برای آموزش مدل.
# این متن شبیه‌سازی یک فایل متنی با حدود 300 خط است.
# در سناریوهای واقعی، این متن از یک فایل بزرگتر (مثلاً یک کتاب) بارگذاری می‌شود.
text = """
جهان هستی پر از شگفتی‌هاست. هر ستاره در آسمان شب، داستانی ناگفته دارد. کهکشان‌ها با میلیاردها ستاره، منظومه‌های شمسی بی‌شمار را در خود جای داده‌اند. زمین، سیاره آبی ما، تنها گوشه‌ای از این عظمت بی‌کران است. حیات در آن به شکل‌های گوناگون جریان دارد. از کوچکترین میکروارگانیسم‌ها تا بزرگترین نهنگ‌ها. همه و همه بخشی از این چرخه شگفت‌انگیز هستند.

علم به ما کمک می‌کند تا این رازها را کشف کنیم. فیزیک، شیمی، زیست‌شناسی، و نجوم. هر کدام دریچه‌ای نو به سوی درک عمیق‌تر جهان می‌گشایند. اکتشافات علمی، مرزهای دانش بشری را جابجا می‌کنند. از نظریه نسبیت انیشتین تا کشف ساختار DNA. این پیشرفت‌ها زندگی ما را متحول کرده‌اند.

فناوری نیز نقش مهمی در این مسیر ایفا می‌کند. از تلسکوپ‌های فضایی قدرتمند تا میکروسکوپ‌های الکترونی پیشرفته. ابزارهایی که به ما امکان مشاهده نادیده‌ها را می‌دهند. هوش مصنوعی و یادگیری ماشینی، حوزه‌های نوظهوری هستند. که پتانسیل عظیمی برای حل مشکلات پیچیده دارند. مدل‌های زبانی بزرگ، نمونه بارز این پیشرفت‌ها هستند.

اما یادگیری فقط به علم و فناوری محدود نمی‌شود. ادبیات، هنر، فلسفه و تاریخ. این‌ها نیز ابعادی از دانش بشری را شکل می‌دهند. که به روح انسان عمق می‌بخشند. مطالعه تاریخ به ما کمک می‌کند تا از گذشته درس بگیریم. و آینده‌ای بهتر بسازیم. ادبیات، دریچه‌ای به سوی احساسات و تجربیات انسانی است. هنر، بیانگر زیبایی و خلاقیت است.

محیط زیست ما در خطر است. تغییرات اقلیمی، آلودگی هوا و آب. از بین رفتن تنوع زیستی. این‌ها چالش‌های بزرگی هستند که با آن‌ها روبرو هستیم. حفاظت از سیاره زمین، مسئولیت همه ماست. با اقدامات کوچک می‌توانیم تاثیرات بزرگی ایجاد کنیم. کاهش مصرف انرژی، بازیافت، و حمایت از انرژی‌های پاک.

آموزش و پرورش، ستون فقرات توسعه هر جامعه است. دسترسی به آموزش با کیفیت برای همه، یک حق اساسی است. مدارس و دانشگاه‌ها باید محیطی برای رشد و شکوفایی استعدادها باشند. یادگیری مادام‌العمر، کلید موفقیت در دنیای امروز است.

سلامتی و تندرستی نیز از اهمیت بالایی برخوردار است. تغذیه سالم، ورزش منظم، و خواب کافی. این‌ها عوامل کلیدی برای یک زندگی سالم هستند. پیشرفت‌های پزشکی، امید به زندگی را افزایش داده‌اند. اما پیشگیری همیشه بهتر از درمان است.

ارتباطات انسانی، پایه و اساس جوامع است. احترام متقابل، همدلی، و درک تفاوت‌ها. این‌ها به ساختن جامعه‌ای صلح‌آمیز کمک می‌کنند. رسانه‌های اجتماعی، ابزارهای قدرتمندی برای ارتباط هستند. اما باید با هوشیاری از آن‌ها استفاده کرد.

اقتصاد جهانی در حال تحول است. دیجیتالی شدن، جهانی شدن، و نوآوری. این‌ها نیروهای محرک اقتصاد امروز هستند. کارآفرینی و خلاقیت، موتور رشد اقتصادی هستند. ایجاد فرصت‌های شغلی جدید، از اولویت‌هاست.

فرهنگ هر جامعه، هویت آن را شکل می‌دهد. آداب و رسوم، زبان، و ارزش‌ها. این‌ها میراثی گرانبها هستند که باید حفظ شوند. تبادل فرهنگی، به غنی‌سازی جوامع کمک می‌کند.

در نهایت، زندگی مجموعه‌ای از تجربیات است. یادگیری از اشتباهات، قدردانی از لحظات خوب. و تلاش برای بهتر شدن. هر روز فرصتی جدید برای رشد و پیشرفت است. با امید به آینده‌ای روشن‌تر.

جهان هستی پر از شگفتی‌هاست. هر ستاره در آسمان شب، داستانی ناگفته دارد. کهکشان‌ها با میلیاردها ستاره، منظومه‌های شمسی بی‌شمار را در خود جای داده‌اند. زمین، سیاره آبی ما، تنها گوشه‌ای از این عظمت بی‌کران است. حیات در آن به شکل‌های گوناگون جریان دارد. از کوچکترین میکروارگانیسم‌ها تا بزرگترین نهنگ‌ها. همه و همه بخشی از این چرخه شگفت‌انگیز هستند.

علم به ما کمک می‌کند تا این رازها را کشف کنیم. فیزیک، شیمی، زیست‌شناسی، و نجوم. هر کدام دریچه‌ای نو به سوی درک عمیق‌تر جهان می‌گشایند. اکتشافات علمی، مرزهای دانش بشری را جابجا می‌کنند. از نظریه نسبیت انیشتین تا کشف ساختار DNA. این پیشرفت‌ها زندگی ما را متحول کرده‌اند.

فناوری نیز نقش مهمی در این مسیر ایفا می‌کند. از تلسکوپ‌های فضایی قدرتمند تا میکروسکوپ‌های الکترونی پیشرفته. ابزارهایی که به ما امکان مشاهده نادیده‌ها را می‌دهند. هوش مصنوعی و یادگیری ماشینی، حوزه‌های نوظهوری هستند. که پتانسیل عظیمی برای حل مشکلات پیچیده دارند. مدل‌های زبانی بزرگ، نمونه بارز این پیشرفت‌ها هستند.

اما یادگیری فقط به علم و فناوری محدود نمی‌شود. ادبیات، هنر، فلسفه و تاریخ. این‌ها نیز ابعادی از دانش بشری را شکل می‌دهند. که به روح انسان عمق می‌بخشند. مطالعه تاریخ به ما کمک می‌کند تا از گذشته درس بگیریم. و آینده‌ای بهتر بسازیم. ادبیات، دریچه‌ای به سوی احساسات و تجربیات انسانی است. هنر، بیانگر زیبایی و خلاقیت است.

محیط زیست ما در خطر است. تغییرات اقلیمی، آلودگی هوا و آب. از بین رفتن تنوع زیستی. این‌ها چالش‌های بزرگی هستند که با آن‌ها روبرو هستیم. حفاظت از سیاره زمین، مسئولیت همه ماست. با اقدامات کوچک می‌توانیم تاثیرات بزرگی ایجاد کنیم. کاهش مصرف انرژی، بازیافت، و حمایت از انرژی‌های پاک.

آموزش و پرورش، ستون فقرات توسعه هر جامعه است. دسترسی به آموزش با کیفیت برای همه، یک حق اساسی است. مدارس و دانشگاه‌ها باید محیطی برای رشد و شکوفایی استعدادها باشند. یادگیری مادام‌العمر، کلید موفقیت در دنیای امروز است.

سلامتی و تندرستی نیز از اهمیت بالایی برخوردار است. تغذیه سالم، ورزش منظم، و خواب کافی. این‌ها عوامل کلیدی برای یک زندگی سالم هستند. پیشرفت‌های پزشکی، امید به زندگی را افزایش داده‌اند. اما پیشگیری همیشه بهتر از درمان است.

ارتباطات انسانی، پایه و اساس جوامع است. احترام متقابل، همدلی، و درک تفاوت‌ها. این‌ها به ساختن جامعه‌ای صلح‌آمیز کمک می‌کنند. رسانه‌های اجتماعی، ابزارهای قدرتمندی برای ارتباط هستند. اما باید با هوشیاری از آن‌ها استفاده کرد.

اقتصاد جهانی در حال تحول است. دیجیتالی شدن، جهانی شدن، و نوآوری. این‌ها نیروهای محرک اقتصاد امروز هستند. کارآفرینی و خلاقیت، موتور رشد اقتصادی هستند. ایجاد فرصت‌های شغلی جدید، از اولویت‌هاست.

فرهنگ هر جامعه، هویت آن را شکل می‌دهد. آداب و رسوم، زبان، و ارزش‌ها. این‌ها میراثی گرانبها هستند که باید حفظ شوند. تبادل فرهنگی، به غنی‌سازی جوامع کمک می‌کند.

در نهایت، زندگی مجموعه‌ای از تجربیات است. یادگیری از اشتباهات، قدردانی از لحظات خوب. و تلاش برای بهتر شدن. هر روز فرصتی جدید برای رشد و پیشرفت است. با امید به آینده‌ای روشن‌تر.

جهان هستی پر از شگفتی‌هاست. هر ستاره در آسمان شب، داستانی ناگفته دارد. کهکشان‌ها با میلیاردها ستاره، منظومه‌های شمسی بی‌شمار را در خود جای داده‌اند. زمین، سیاره آبی ما، تنها گوشه‌ای از این عظمت بی‌کران است. حیات در آن به شکل‌های گوناگون جریان دارد. از کوچکترین میکروارگانیسم‌ها تا بزرگترین نهنگ‌ها. همه و همه بخشی از این چرخه شگفت‌انگیز هستند.

علم به ما کمک می‌کند تا این رازها را کشف کنیم. فیزیک، شیمی، زیست‌شناسی، و نجوم. هر کدام دریچه‌ای نو به سوی درک عمیق‌تر جهان می‌گشایند. اکتشافات علمی، مرزهای دانش بشری را جابجا می‌کنند. از نظریه نسبیت انیشتین تا کشف ساختار DNA. این پیشرفت‌ها زندگی ما را متحول کرده‌اند.

فناوری نیز نقش مهمی در این مسیر ایفا می‌کند. از تلسکوپ‌های فضایی قدرتمند تا میکروسکوپ‌های الکترونی پیشرفته. ابزارهایی که به ما امکان مشاهده نادیده‌ها را می‌دهند. هوش مصنوعی و یادگیری ماشینی، حوزه‌های نوظهوری هستند. که پتانسیل عظیمی برای حل مشکلات پیچیده دارند. مدل‌های زبانی بزرگ، نمونه بارز این پیشرفت‌ها هستند.

اما یادگیری فقط به علم و فناوری محدود نمی‌شود. ادبیات، هنر، فلسفه و تاریخ. این‌ها نیز ابعادی از دانش بشری را شکل می‌دهند. که به روح انسان عمق می‌بخشند. مطالعه تاریخ به ما کمک می‌کند تا از گذشته درس بگیریم. و آینده‌ای بهتر بسازیم. ادبیات، دریچه‌ای به سوی احساسات و تجربیات انسانی است. هنر، بیانگر زیبایی و خلاقیت است.

محیط زیست ما در خطر است. تغییرات اقلیمی، آلودگی هوا و آب. از بین رفتن تنوع زیستی. این‌ها چالش‌های بزرگی هستند که با آن‌ها روبرو هستیم. حفاظت از سیاره زمین، مسئولیت همه ماست. با اقدامات کوچک می‌توانیم تاثیرات بزرگی ایجاد کنیم. کاهش مصرف انرژی، بازیافت، و حمایت از انرژی‌های پاک.

آموزش و پرورش، ستون فقرات توسعه هر جامعه است. دسترسی به آموزش با کیفیت برای همه، یک حق اساسی است. مدارس و دانشگاه‌ها باید محیطی برای رشد و شکوفایی استعدادها باشند. یادگیری مادام‌العمر، کلید موفقیت در دنیای امروز است.

سلامتی و تندرستی نیز از اهمیت بالایی برخوردار است. تغذیه سالم، ورزش منظم، و خواب کافی. این‌ها عوامل کلیدی برای یک زندگی سالم هستند. پیشرفت‌های پزشکی، امید به زندگی را افزایش داده‌اند. اما پیشگیری همیشه بهتر از درمان است.

ارتباطات انسانی، پایه و اساس جوامع است. احترام متقابل، همدلی، و درک تفاوت‌ها. این‌ها به ساختن جامعه‌ای صلح‌آمیز کمک می‌کنند. رسانه‌های اجتماعی، ابزارهای قدرتمندی برای ارتباط هستند. اما باید با هوشیاری از آن‌ها استفاده کرد.

اقتصاد جهانی در حال تحول است. دیجیتالی شدن، جهانی شدن، و نوآوری. این‌ها نیروهای محرک اقتصاد امروز هستند. کارآفرینی و خلاقیت، موتور رشد اقتصادی هستند. ایجاد فرصت‌های شغلی جدید، از اولویت‌هاست.

فرهنگ هر جامعه، هویت آن را شکل می‌دهد. آداب و رسوم، زبان، و ارزش‌ها. این‌ها میراثی گرانبها هستند که باید حفظ شوند. تبادل فرهنگی، به غنی‌سازی جوامع کمک می‌کند.

در نهایت، زندگی مجموعه‌ای از تجربیات است. یادگیری از اشتباهات، قدردانی از لحظات خوب. و تلاش برای بهتر شدن. هر روز فرصتی جدید برای رشد و پیشرفت است. با امید به آینده‌ای روشن‌تر.

جهان هستی پر از شگفتی‌هاست. هر ستاره در آسمان شب، داستانی ناگفته دارد. کهکشان‌ها با میلیاردها ستاره، منظومه‌های شمسی بی‌شمار را در خود جای داده‌اند. زمین، سیاره آبی ما، تنها گوشه‌ای از این عظمت بی‌کران است. حیات در آن به شکل‌های گوناگون جریان دارد. از کوچکترین میکروارگانیسم‌ها تا بزرگترین نهنگ‌ها. همه و همه بخشی از این چرخه شگفت‌انگیز هستند.

علم به ما کمک می‌کند تا این رازها را کشف کنیم. فیزیک، شیمی، زیست‌شناسی، و نجوم. هر کدام دریچه‌ای نو به سوی درک عمیق‌تر جهان می‌گشایند. اکتشافات علمی، مرزهای دانش بشری را جابجا می‌کنند. از نظریه نسبیت انیشتین تا کشف ساختار DNA. این پیشرفت‌ها زندگی ما را متحول کرده‌اند.

فناوری نیز نقش مهمی در این مسیر ایفا می‌کند. از تلسکوپ‌های فضایی قدرتمند تا میکروسکوپ‌های الکترونی پیشرفته. ابزارهایی که به ما امکان مشاهده نادیده‌ها را می‌دهند. هوش مصنوعی و یادگیری ماشینی، حوزه‌های نوظهوری هستند. که پتانسیل عظیمی برای حل مشکلات پیچیده دارند. مدل‌های زبانی بزرگ، نمونه بارز این پیشرفت‌ها هستند.

اما یادگیری فقط به علم و فناوری محدود نمی‌شود. ادبیات، هنر، فلسفه و تاریخ. این‌ها نیز ابعادی از دانش بشری را شکل می‌دهند. که به روح انسان عمق می‌بخشند. مطالعه تاریخ به ما کمک می‌کند تا از گذشته درس بگیریم. و آینده‌ای بهتر بسازیم. ادبیات، دریچه‌ای به سوی احساسات و تجربیات انسانی است. هنر، بیانگر زیبایی و خلاقیت است.

محیط زیست ما در خطر است. تغییرات اقلیمی، آلودگی هوا و آب. از بین رفتن تنوع زیستی. این‌ها چالش‌های بزرگی هستند که با آن‌ها روبرو هستیم. حفاظت از سیاره زمین، مسئولیت همه ماست. با اقدامات کوچک می‌توانیم تاثیرات بزرگی ایجاد کنیم. کاهش مصرف انرژی، بازیافت، و حمایت از انرژی‌های پاک.

آموزش و پرورش، ستون فقرات توسعه هر جامعه است. دسترسی به آموزش با کیفیت برای همه، یک حق اساسی است. مدارس و دانشگاه‌ها باید محیطی برای رشد و شکوفایی استعدادها باشند. یادگیری مادام‌العمر، کلید موفقیت در دنیای امروز است.

سلامتی و تندرستی نیز از اهمیت بالایی برخوردار است. تغذیه سالم، ورزش منظم، و خواب کافی. این‌ها عوامل کلیدی برای یک زندگی سالم هستند. پیشرفت‌های پزشکی، امید به زندگی را افزایش داده‌اند. اما پیشگیری همیشه بهتر از درمان است.

ارتباطات انسانی، پایه و اساس جوامع است. احترام متقابل، همدلی، و درک تفاوت‌ها. این‌ها به ساختن جامعه‌ای صلح‌آمیز کمک می‌کنند. رسانه‌های اجتماعی، ابزارهای قدرتمندی برای ارتباط هستند. اما باید با هوشیاری از آن‌ها استفاده کرد.

اقتصاد جهانی در حال تحول است. دیجیتالی شدن، جهانی شدن، و نوآوری. این‌ها نیروهای محرک اقتصاد امروز هستند. کارآفرینی و خلاقیت، موتور رشد اقتصادی هستند. ایجاد فرصت‌های شغلی جدید، از اولویت‌هاست.

فرهنگ هر جامعه، هویت آن را شکل می‌دهد. آداب و رسوم، زبان، و ارزش‌ها. این‌ها میراثی گرانبها هستند که باید حفظ شوند. تبادل فرهنگی، به غنی‌سازی جوامع کمک می‌کند.

در نهایت، زندگی مجموعه‌ای از تجربیات است. یادگیری از اشتباهات، قدردانی از لحظات خوب. و تلاش برای بهتر شدن. هر روز فرصتی جدید برای رشد و پیشرفت است. با امید به آینده‌ای روشن‌تر.

جهان هستی پر از شگفتی‌هاست. هر ستاره در آسمان شب، داستانی ناگفته دارد. کهکشان‌ها با میلیاردها ستاره، منظومه‌های شمسی بی‌شمار را در خود جای داده‌اند. زمین، سیاره آبی ما، تنها گوشه‌ای از این عظمت بی‌کران است. حیات در آن به شکل‌های گوناگون جریان دارد. از کوچکترین میکروارگانیسم‌ها تا بزرگترین نهنگ‌ها. همه و همه بخشی از این چرخه شگفت‌انگیز هستند.

علم به ما کمک می‌کند تا این رازها را کشف کنیم. فیزیک، شیمی، زیست‌شناسی، و نجوم. هر کدام دریچه‌ای نو به سوی درک عمیق‌تر جهان می‌گشایند. اکتشافات علمی، مرزهای دانش بشری را جابجا می‌کنند. از نظریه نسبیت انیشتین تا کشف ساختار DNA. این پیشرفت‌ها زندگی ما را متحول کرده‌اند.

فناوری نیز نقش مهمی در این مسیر ایفا می‌کند. از تلسکوپ‌های فضایی قدرتمند تا میکروسکوپ‌های الکترونی پیشرفته. ابزارهایی که به ما امکان مشاهده نادیده‌ها را می‌دهند. هوش مصنوعی و یادگیری ماشینی، حوزه‌های نوظوری هستند. که پتانسیل عظیمی برای حل مشکلات پیچیده دارند. مدل‌های زبانی بزرگ، نمونه بارز این پیشرفت‌ها هستند.

اما یادگیری فقط به علم و فناوری محدود نمی‌شود. ادبیات، هنر، فلسفه و تاریخ. این‌ها نیز ابعادی از دانش بشری را شکل می‌دهند. که به روح انسان عمق می‌بخشند. مطالعه تاریخ به ما کمک می‌کند تا از گذشته درس بگیریم. و آینده‌ای بهتر بسازیم. ادبیات، دریچه‌ای به سوی احساسات و تجربیات انسانی است. هنر، بیانگر زیبایی و خلاقیت است.

محیط زیست ما در خطر است. تغییرات اقلیمی، آلودگی هوا و آب. از بین رفتن تنوع زیستی. این‌ها چالش‌های بزرگی هستند که با آن‌ها روبرو هستیم. حفاظت از سیاره زمین، مسئولیت همه ماست. با اقدامات کوچک می‌توانیم تاثیرات بزرگی ایجاد کنیم. کاهش مصرف انرژی، بازیافت، و حمایت از انرژی‌های پاک.

آموزش و پرورش، ستون فقرات توسعه هر جامعه است. دسترسی به آموزش با کیفیت برای همه، یک حق اساسی است. مدارس و دانشگاه‌ها باید محیطی برای رشد و شکوفایی استعدادها باشند. یادگیری مادام‌العمر، کلید موفقیت در دنیای امروز است.

سلامتی و تندرستی نیز از اهمیت بالایی برخوردار است. تغذیه سالم، ورزش منظم، و خواب کافی. این‌ها عوامل کلیدی برای یک زندگی سالم هستند. پیشرفت‌های پزشکی، امید به زندگی را افزایش داده‌اند. اما پیشگیری همیشه بهتر از درمان است.

ارتباطات انسانی، پایه و اساس جوامع است. احترام متقابل، همدلی، و درک تفاوت‌ها. این‌ها به ساختن جامعه‌ای صلح‌آمیز کمک می‌کنند. رسانه‌های اجتماعی، ابزارهای قدرتمندی برای ارتباط هستند. اما باید با هوشیاری از آن‌ها استفاده کرد.

اقتصاد جهانی در حال تحول است. دیجیتالی شدن، جهانی شدن، و نوآوری. این‌ها نیروهای محرک اقتصاد امروز هستند. کارآفرینی و خلاقیت، موتور رشد اقتصادی هستند. ایجاد فرصت‌های شغلی جدید، از اولویت‌هاست.

فرهنگ هر جامعه، هویت آن را شکل می‌دهد. آداب و رسوم، زبان، و ارزش‌ها. این‌ها میراثی گرانبها هستند که باید حفظ شوند. تبادل فرهنگی، به غنی‌سازی جوامع کمک می‌کند.

در نهایت، زندگی مجموعه‌ای از تجربیات است. یادگیری از اشتباهات، قدردانی از لحظات خوب. و تلاش برای بهتر شدن. هر روز فرصتی جدید برای رشد و پیشرفت است. با امید به آینده‌ای روشن‌تر.
"""

# تبدیل تمام متن به حروف کوچک برای یکسان‌سازی
text = text.lower()
print(f"\nمتن اصلی برای آموزش (بخش اول):\n{text[:500]}...\n") # نمایش بخشی از متن

# --- توکن‌سازی (Tokenization) ---
# توکن‌سازی فرآیند تبدیل متن به دنباله‌ای از اعداد است.
# هر کلمه منحصر به فرد یک عدد یکتا دریافت می‌کند.
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text]) # مدل توکن‌ساز را روی متن آموزش می‌دهیم تا کلمات را بشناسد
word_index = tokenizer.word_index # دیکشنری کلمات به اعداد (ایندکس‌ها)
print(f"تعداد کلمات منحصر به فرد در واژه‌نامه: {len(word_index)}\n")

# تعداد کل کلمات منحصر به فرد + 1 (برای کلماتی که در واژه‌نامه نیستند یا برای padding)
total_words = len(word_index) + 1
print(f"تعداد کل کلمات منحصر به فرد در واژه‌نامه (با احتساب صفر): {total_words}\n")

# --- ساخت دنباله‌های ورودی و خروجی برای آموزش ---
# برای آموزش مدل، دنباله‌هایی از کلمات (n-grams) ایجاد می‌کنیم.
input_sequences = []
for line in text.split('\n'):
    if line.strip() == "": # خطوط خالی را نادیده می‌گیریم
        continue
    token_list = tokenizer.texts_to_sequences([line])[0] # هر خط را به دنباله‌ای از اعداد تبدیل می‌کنیم
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1] # دنباله n-gram (از ابتدای خط تا کلمه فعلی)
        input_sequences.append(n_gram_sequence)

print(f"تعداد دنباله‌های ورودی برای آموزش: {len(input_sequences)}\n")
print(f"نمونه‌ای از دنباله‌های ورودی (اعداد):\n{input_sequences[:5]}\n")

# --- هم‌اندازه‌سازی دنباله‌ها (Padding) و تفکیک X و Y ---
# شبکه‌های عصبی نیاز دارند که ورودی‌ها هم‌اندازه باشند.
# بنابراین، دنباله‌های کوتاه‌تر را با صفر در ابتدا پر می‌کنیم (pre-padding).
max_sequence_len = max([len(x) for x in input_sequences])
print(f"حداکثر طول دنباله در داده‌های آموزشی: {max_sequence_len}\n")

padded_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                          maxlen=max_sequence_len,
                                                                          padding='pre'))

# X (ورودی): تمام کلمات دنباله به جز آخرین کلمه
X = padded_sequences[:, :-1]
# labels (خروجی): فقط آخرین کلمه در هر دنباله (کلمه‌ای که می‌خواهیم پیش‌بینی کنیم)
labels = padded_sequences[:, -1]

# تبدیل خروجی (labels) به فرمت one-hot encoding
# چون پیش‌بینی کلمه یک مسئله دسته‌بندی است، خروجی‌ها باید به صورت one-hot باشند.
y = to_categorical(labels, num_classes=total_words)

print(f"ابعاد ورودی (X) برای مدل: {X.shape}") # (تعداد نمونه‌ها, طول دنباله ورودی)
print(f"ابعاد خروجی (y) برای مدل (one-hot): {y.shape}") # (تعداد نمونه‌ها, تعداد کل کلمات در واژه‌نامه)

# --- ۲. ساخت مدل GRU ---
# معماری مدل GRU را با پارامترهای جدید تعریف می‌کنیم.
model = Sequential()
# لایه Embedding: ابعاد بردار کلمه را به 256 افزایش می‌دهیم.
# total_words: اندازه واژه‌نامه
# 256: ابعاد بردار کلمه (embedding_dim)
# input_length: طول دنباله ورودی به لایه GRU (max_sequence_len - 1)
model.add(Embedding(total_words, 256, input_length=max_sequence_len-1))
# لایه GRU اول: 512 واحد حافظه.
# return_sequences=True: خروجی هر گام زمانی را برای لایه GRU بعدی ارسال می‌کند.
model.add(GRU(512, return_sequences=True))
# لایه GRU دوم: 512 واحد حافظه.
# return_sequences=False: فقط خروجی آخرین گام زمانی را برای لایه Dense ارسال می‌کند.
model.add(GRU(512, return_sequences=False))
# لایه Dense: لایه خروجی با تابع فعال‌سازی softmax.
# total_words: تعداد نورون‌ها برابر با تعداد کل کلمات در واژه‌نامه است.
model.add(Dense(total_words, activation='softmax'))

# کامپایل کردن مدل: تعریف تابع هزینه، بهینه‌ساز و معیار ارزیابی
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# --- ۳. آموزش مدل ---
# تعداد epochs را به 200 افزایش می‌دهیم.
# این مرحله ممکن است زمان‌بر باشد، به خصوص اگر از GPU استفاده نکنید.
print("\nشروع آموزش مدل GRU...")
history = model.fit(X, y, epochs=200, verbose=1) # verbose=1 برای نمایش جزئیات آموزش

print("\nآموزش مدل به پایان رسید.")
print(f"دقت نهایی مدل در آموزش: {history.history['accuracy'][-1]:.4f}")
print(f"خطای نهایی مدل در آموزش: {history.history['loss'][-1]:.4f}")

# --- ۴. تولید متن با مدل آموزش‌دیده ---
# تابعی برای تولید کلمات بعدی بر اساس یک متن اولیه (seed text)
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    generated_text = seed_text
    for _ in range(next_words):
        # تبدیل متن اولیه به دنباله عددی
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        # هم‌اندازه‌سازی دنباله برای ورودی مدل
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list],
                                                                  maxlen=max_sequence_len-1,
                                                                  padding='pre')

        # پیش‌بینی احتمال هر کلمه به عنوان کلمه بعدی
        predicted_probs = model.predict(token_list, verbose=0)[0]
        # انتخاب کلمه‌ای با بالاترین احتمال (argmax)
        predicted_word_index = np.argmax(predicted_probs)

        output_word = ""
        # پیدا کردن کلمه متناظر با ایندکس پیش‌بینی شده
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        # اگر کلمه‌ای پیدا نشد (مثلاً ایندکس 0 که برای padding است)
        if output_word == "":
            output_word = "<unk>" # کلمه ناشناخته

        generated_text += " " + output_word
    return generated_text

# --- امتحان تولید متن ---
# یک کلمه یا عبارت اولیه بدهید تا مدل از آنجا شروع به تولید متن کند.
# سعی کنید از کلماتی استفاده کنید که در متن آموزشی وجود دارند.
seed_text_input_1 = "جهان هستی"
num_words_to_generate = 20 # تعداد کلماتی که می‌خواهید مدل تولید کند

print(f"\nشروع تولید متن با عبارت اولیه: '{seed_text_input_1}'")
generated_sentence_1 = generate_text(seed_text_input_1, num_words_to_generate, model, max_sequence_len, tokenizer)
print(f"\nمتن تولید شده توسط مدل:\n{generated_sentence_1}")

# مثال دیگر
seed_text_input_2 = "علم به ما"
generated_sentence_2 = generate_text(seed_text_input_2, num_words_to_generate, model, max_sequence_len, tokenizer)
print(f"\nمتن تولید شده دیگر توسط مدل:\n{generated_sentence_2}")

# مثال سوم
seed_text_input_3 = "فناوری نیز"
generated_sentence_3 = generate_text(seed_text_input_3, num_words_to_generate, model, max_sequence_len, tokenizer)
print(f"\nمتن تولید شده سوم توسط مدل:\n{generated_sentence_3}")


کتابخانه‌ها با موفقیت وارد شدند.

متن اصلی برای آموزش (بخش اول):

جهان هستی پر از شگفتی‌هاست. هر ستاره در آسمان شب، داستانی ناگفته دارد. کهکشان‌ها با میلیاردها ستاره، منظومه‌های شمسی بی‌شمار را در خود جای داده‌اند. زمین، سیاره آبی ما، تنها گوشه‌ای از این عظمت بی‌کران است. حیات در آن به شکل‌های گوناگون جریان دارد. از کوچکترین میکروارگانیسم‌ها تا بزرگترین نهنگ‌ها. همه و همه بخشی از این چرخه شگفت‌انگیز هستند.

علم به ما کمک می‌کند تا این رازها را کشف کنیم. فیزیک، شیمی، زیست‌شناسی، و نجوم. هر کدام دریچه‌ای نو به سوی درک عمیق‌تر جهان می‌گشایند. اکتشافات علمی، مرزه...

تعداد کلمات منحصر به فرد در واژه‌نامه: 293

تعداد کل کلمات منحصر به فرد در واژه‌نامه (با احتساب صفر): 294

تعداد دنباله‌های ورودی برای آموزش: 2310

نمونه‌ای از دنباله‌های ورودی (اعداد):
[[26, 61], [26, 61, 62], [26, 61, 62, 2], [26, 61, 62, 2, 63], [26, 61, 62, 2, 63, 12]]

حداکثر طول دنباله در داده‌های آموزشی: 59

ابعاد ورودی (X) برای مدل: (2310, 58)
ابعاد خروجی (y) برای مدل (one-hot): (2310, 294)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


شروع آموزش مدل GRU...
Epoch 1/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.0486 - loss: 5.4661
Epoch 2/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.1834 - loss: 3.8577
Epoch 3/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5599 - loss: 1.4745
Epoch 4/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8566 - loss: 0.5255
Epoch 5/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9851 - loss: 0.1220
Epoch 6/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9975 - loss: 0.0491
Epoch 7/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9993 - loss: 0.0280
Epoch 8/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9993 - loss: 0.0200
Epoch 9/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9992 - loss: 0.0136
Epoch 10/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9992 - loss: 0.0110
Epoch 11/200
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9996 - loss: 0.0091
Epoch 12/200
73/73 ━━━━━━━━━━